Analice el dataset e indique algunos estadísticos descriptivos (numero de canciones,
valores promedio de los atributos para cada canción, entre otros).

In [113]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('spotify.csv')
display(df.head())

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [114]:
import pandas as pd

# --- 1. Cargar y limpiar el dataset ---
df = pd.read_csv("spotify.csv")
df = df.drop_duplicates()
df = df.dropna()
df = df[~(df == '').any(axis=1)]

# Verificar y eliminar duplicados por track_id
df = df.drop_duplicates(subset='track_id')
df = df.drop_duplicates(subset=['track_name', 'artists', 'track_genre']).reset_index(drop=True)

# --- 2. Resumen general del dataset ---
total_canciones = len(df)
total_generos = df['track_genre'].nunique()
duracion_promedio = df['duration_ms'].mean() / 1000  # en segundos
porcentaje_explicitas = df['explicit'].mean() * 100

print("RESUMEN GENERAL DEL DATASET")
print(f"- Total de canciones: {total_canciones}")
print(f"- Total de géneros únicos: {total_generos}")
print(f"- Duración promedio (segundos): {round(duracion_promedio, 2)}")
print(f"- Porcentaje de canciones explícitas: {round(porcentaje_explicitas, 2)}%\n")

# --- 3. Análisis de atributos por género ---
features = ['danceability', 'energy', 'acousticness', 'instrumentalness', 'valence']
genre_means = df.groupby('track_genre')[features].mean()

print("TOP Y BOTTOM 5 POR ATRIBUTO:\n")
for feature in features:
    print(f"{feature.upper()}")
    print("Top 5:")
    print(genre_means[feature].sort_values(ascending=False).head(5), "\n")
    print("Bottom 5:")
    print(genre_means[feature].sort_values().head(5), "\n")

# --- 4. Tabla completa: promedio por género y atributo ---
tabla_resumen = genre_means.reset_index()
print("TABLA DE PROMEDIOS POR GÉNERO:")
print(tabla_resumen.head(10))  # Muestra solo los primeros 10 para evitar exceso


# --- 5. Promedios generales por atributo ---
features = ['danceability', 'energy', 'acousticness', 'instrumentalness',
            'liveness', 'valence', 'tempo', 'speechiness', 'loudness']
promedios_generales = df[features].mean().round(3)

promedios_generales


RESUMEN GENERAL DEL DATASET
- Total de canciones: 81885
- Total de géneros únicos: 113
- Duración promedio (segundos): 231.34
- Porcentaje de canciones explícitas: 8.58%

TOP Y BOTTOM 5 POR ATRIBUTO:

DANCEABILITY
Top 5:
track_genre
kids              0.781198
chicago-house     0.766536
latino            0.755441
reggaeton         0.743247
minimal-techno    0.732811
Name: danceability, dtype: float64 

Bottom 5:
track_genre
sleep          0.167355
grindcore      0.272635
black-metal    0.295886
iranian        0.299983
opera          0.305819
Name: danceability, dtype: float64 

ENERGY
Top 5:
track_genre
death-metal    0.931851
grindcore      0.925971
happy          0.909754
metalcore      0.901839
hardstyle      0.899980
Name: energy, dtype: float64 

Bottom 5:
track_genre
classical    0.202154
new-age      0.211647
ambient      0.235512
opera        0.290388
disney       0.297300
Name: energy, dtype: float64 

ACOUSTICNESS
Top 5:
track_genre
classical    0.915637
romance      0.866005


,0
danceability,0.560
energy,0.636
acousticness,0.329
instrumentalness,0.184
liveness,0.220
valence,0.464
tempo,122.150
speechiness,0.089
loudness,-8.580


In [115]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# --- 1. Cargar y limpiar datos ---
df = pd.read_csv("spotify.csv")
df = df.drop_duplicates()
df = df.dropna()
df = df[~(df == '').any(axis=1)]
df = df.drop_duplicates(subset='track_id')
df = df.drop_duplicates(subset=['track_name', 'artists', 'track_genre']).reset_index(drop=True)

# --- 2. Seleccionar atributos relevantes ---
features = ['danceability', 'energy', 'acousticness', 'instrumentalness',
            'liveness', 'valence', 'tempo', 'speechiness', 'loudness']

# --- 3. Escalar los datos ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

# --- 4. Aplicar k-NN con métrica de coseno ---
knn = NearestNeighbors(n_neighbors=11, metric='cosine')  # 10 vecinos + la canción misma
knn.fit(X_scaled)

# --- 5. Buscar canciones similares a las primeras 5 ---
distances, indices = knn.kneighbors(X_scaled[:5])

# --- 6. Mostrar resultados ---
for i, neighbors in enumerate(indices):
    print(f"\nCanción: {df.iloc[i]['track_name']} - {df.iloc[i]['artists']}")
    print("Canciones más similares:")
    for rank, idx in enumerate(neighbors[1:], start=1):  # omitimos la canción original
        similar_track = df.iloc[idx]
        distance = distances[i][rank]
        print(f" - {similar_track['track_name']} - {similar_track['artists']} | Distancia coseno: {round(distance, 4)}")


Canción: Comedy - Gen Hoshino
Canciones más similares:
 - Skylight - Gramatik | Distancia coseno: 0.0445
 - Look For The Good (Single Version) - Jason Mraz | Distancia coseno: 0.0665
 - That Certain Female - Charlie Feathers | Distancia coseno: 0.0666
 - Monde de fous - Danakil | Distancia coseno: 0.0675
 - Pop Virus - Gen Hoshino | Distancia coseno: 0.0701
 - Blink 182 - Trille | Distancia coseno: 0.0705
 - Mi Eden de Tristeza - Porta;Pumpkin | Distancia coseno: 0.0714
 - The Hard Stuff - The Herbaliser | Distancia coseno: 0.0791
 - The Gambler - Busy Signal | Distancia coseno: 0.0844
 - Bu Aşk Olur Mu - MFÖ | Distancia coseno: 0.0873

Canción: Ghost - Acoustic - Ben Woodward
Canciones más similares:
 - Viejo Ciego (feat. Antonio Agri & Esteban Morgado) [En Directo] - Roberto Goyeneche;Antonio Agri;Esteban Morgado | Distancia coseno: 0.0053
 - Carmen, Act II: No.15 Quintette : Nous avons en tête une affaire (Le Dancaïro/Frasquita/Mercédès/Le Remendado/Carmen) - Georges Bizet;Nicolas 

La distancia del coseno mide qué tan parecida es la forma del perfil musical entre canciones, sin importar cuánto más fuerte, más suave o más larga es una respecto a otra.

Es ideal cuando:

Importa más la relación entre atributos que sus valores absolutos.

Estás trabajando con vectores de características escaladas o normalizadas.

In [116]:
### Este código se utilizó para buscar si estaban las canciones en la BD
def buscar_canciones(df, nombre_parcial):
    resultado = df[df['track_name'].str.lower().str.contains(nombre_parcial.lower())]
    if resultado.empty:
        print(f"No se encontraron canciones que contengan: '{nombre_parcial}'")
    else:
        return resultado[['track_name', 'track_id', 'track_genre', 'artists']].reset_index(drop=True)


buscar_canciones(df, "Re:Re:")

,track_name,track_id,track_genre,artists
0,Re:Re:,0WaaPFt4Qy8sVfxKz43bCD,anime,ASIAN KUNG-FU GENERATION
1,Re:Re: (Single ver.),3rxLb8cYhEnv3ZEoluDdpA,emo,ASIAN KUNG-FU GENERATION


In [117]:
def canciones_similares_a_grupo(df, canciones_base, n_resultados=10):
    indices = []
    for nombre in canciones_base:
        fila = df[df['track_id'].str.lower() == nombre.lower()]
        if not fila.empty:
            idx = fila.index[0]
            indices.append(idx)
            print("Canción seleccionada:")
            print(df.loc[idx])
        else:
            print(f"Canción no encontrada: {nombre}")

    if not indices:
        return None, "No se encontraron canciones válidas."

    # Calcular vector promedio del grupo
    vector_prom = X_scaled[indices].mean(axis=0).reshape(1, -1)

    # Buscar canciones similares con distancias
    distances, idxs = knn.kneighbors(vector_prom)
    vecinos = [i for i in idxs[0] if i not in indices][:n_resultados]
    distancias = [d for i, d in zip(idxs[0], distances[0]) if i not in indices][:n_resultados]

    # Crear DataFrame con resultados y distancias
    canciones_similares = df.iloc[vecinos][['track_name', 'artists', 'track_id', 'track_genre']].copy()
    canciones_similares['distancia_coseno'] = distancias

    return canciones_similares.reset_index(drop=True), vector_prom

In [118]:
from sklearn.neighbors import KNeighborsRegressor

def predecir_rating(df, vector_base, ratings_usuario, canciones_similares):
    # Obtener atributos de las canciones similares
    idx_similares = []
    print(len(df))
    for nombre in canciones_similares['track_id']:
        idx = df[df['track_id'] == nombre].index[0]
        idx_similares.append(idx)

    X_train = X_scaled[idx_similares]
    y_train = ratings_usuario[:len(X_train)]

    # Entrenar modelo de regresión k-NN con distancia coseno
    knn_reg = KNeighborsRegressor(n_neighbors=10, metric='cosine')
    knn_reg.fit(X_train, y_train)

    # Predecir rating para la canción base
    rating_predicho = knn_reg.predict(vector_base)[0]
    return round(rating_predicho, 2)

In [119]:
# Rafa: Dar canciones base

# '0WaaPFt4Qy8sVfxKz43bCD' Re:Re: Asian kung fu generation (4)

canciones_base = ['0WaaPFt4Qy8sVfxKz43bCD']
similares, vector_prom = canciones_similares_a_grupo(df, canciones_base)
print()
print(similares)


Canción seleccionada:
Unnamed: 0                              5259
track_id              0WaaPFt4Qy8sVfxKz43bCD
artists             ASIAN KUNG-FU GENERATION
album_name                              ソルファ
track_name                            Re:Re:
popularity                                59
duration_ms                           228760
explicit                               False
danceability                           0.537
energy                                 0.864
key                                        4
loudness                              -5.049
mode                                       1
speechiness                           0.0369
acousticness                        0.000318
instrumentalness                       0.123
liveness                               0.108
valence                                0.727
tempo                                153.035
time_signature                             4
track_genre                            anime
Name: 4115, dtype: object

      

In [120]:

# Paso 2: Evaluar canciones similares manualmente (1 a 5)
ratings_usuario = [4, 2, 4, 5, 4, 4, 3, 4, 4, 3]  # '0WaaPFt4Qy8sVfxKz43bCD' Asian kung fu generation

# Paso 3: Predecir rating de la canción base (en este caso el promedio del grupo)
rating_estimado = predecir_rating(df, vector_prom, ratings_usuario, similares)
print(f"Rating predicho para el grupo: {rating_estimado}")

81885
Rating predicho para el grupo: 3.7


In [121]:
# Jacob: Dar canciones base

# '0WaaPFt4Qy8sVfxKz43bCD' Asian kung fu generation
# '7qqnbgkCwchM2CZnATg4nX' We know the way - Moana (4)

canciones_base = ['7qqnbgkCwchM2CZnATg4nX']
similares, vector_prom = canciones_similares_a_grupo(df, canciones_base)
print()
print(similares)

Canción seleccionada:
Unnamed: 0                                                      98101
track_id                                       7qqnbgkCwchM2CZnATg4nX
artists                              Lin-Manuel Miranda;Opetaia Foa'i
album_name          Moana (Original Motion Picture Soundtrack/Delu...
track_name                                   We Know The Way - Finale
popularity                                                         36
duration_ms                                                     69120
explicit                                                        False
danceability                                                    0.495
energy                                                          0.839
key                                                                 9
loudness                                                       -8.744
mode                                                                1
speechiness                                                    0.052

In [122]:
# User 2: Evaluar canciones similares manualmente (1 a 5)
ratings_usuario = [4, 3, 2, 5, 3, 4, 4, 2, 2, 5]  # Moana

# Paso 3: Predecir rating de la canción base (en este caso el promedio del grupo)
rating_estimado = predecir_rating(df, vector_prom, ratings_usuario, similares)
print(f"Rating predicho para el grupo: {rating_estimado}")

81885
Rating predicho para el grupo: 3.4


In [123]:
# Seba: Dar canciones base

# '0WaaPFt4Qy8sVfxKz43bCD' Asian kung fu generation (4)
# '7qqnbgkCwchM2CZnATg4nX' We know the way - Moana (3)
# '6IicYKDzPBN2VDClsjxR9x' I'm not Angry - Elvis Costello (4)

canciones_base = ['6IicYKDzPBN2VDClsjxR9x']
similares, vector_prom = canciones_similares_a_grupo(df, canciones_base)
print()
print(similares)

Canción seleccionada:
Unnamed: 0                           82922
track_id            6IicYKDzPBN2VDClsjxR9x
artists                     Elvis Costello
album_name                  My Aim Is True
track_name                   I'm Not Angry
popularity                              22
duration_ms                         179320
explicit                             False
danceability                         0.547
energy                               0.823
key                                      1
loudness                            -8.579
mode                                     0
speechiness                         0.0693
acousticness                         0.159
instrumentalness                     0.119
liveness                             0.325
valence                              0.729
tempo                               139.38
time_signature                           4
track_genre                      power-pop
Name: 62639, dtype: object

                      track_name               

In [124]:
# User 3: Evaluar canciones similares manualmente (1 a 5)
ratings_usuario = [5, 4, 2, 5, 3, 4, 4, 3, 5, 4]  # I'm not angry - Elvis Costello

# Paso 3: Predecir rating de la canción base (en este caso el promedio del grupo)
rating_estimado = predecir_rating(df, vector_prom, ratings_usuario, similares)
print(f"Rating predicho para el grupo: {rating_estimado}")

81885
Rating predicho para el grupo: 3.9
